<a href="https://colab.research.google.com/github/tiago15anjos/Agrega-o_ICJ_2010_2023/blob/main/ICJ_Municipio_Ano_2010_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# === 0. Carregar o arquivo ================================================
df = pd.read_excel("ICJ_CNPQ_2010_2023_LIMPO.xlsx")

# === 1. Definições iniciais ===============================================
lvl_city_year = [
    "Regiao_Geografica", "Unidade_Federacao", "Nome_Municipio",
    "Codigo_IBGE", "Ano", "Sigla_UF", "Pais"
]
lvl_city_year_inst = lvl_city_year + ["Instituição"]

# Métricas numéricas que terão média e soma
value_cols = [
    "Valor_Rs", "Valor_USS", "Benef_Modal/Ano",
    "Bolsa/Ano", "Auxílio/Ano"
]

# Função auxiliar para renomear colunas multi‑nível
def flatten_cols(df_grouped):
    df_grouped.columns = [
        f"{col}_{func if func!='size' else ''}".rstrip("_")
        for col, func in df_grouped.columns
    ]
    return df_grouped.reset_index()

# === 2. FATO MUNICÍPIO + ANO =============================================
agg_city_year = {c: ["mean", "sum"] for c in value_cols}
agg_city_year["Codigo_IBGE"] = "size"               # contagem de linhas
agg_city_year["Instituição"] = (
    lambda x: "; ".join(sorted(x.unique()))         # lista de instituições*
)

fato_municipio_ano = (
    df.groupby(lvl_city_year, dropna=False)
      .agg(agg_city_year)
      .rename(columns={"Codigo_IBGE": "Qtd_Bolsistas"})
      .pipe(flatten_cols)
)

fato_municipio_ano.to_excel("fato_municipio_ano.xlsx", index=False)

# === 3. FATO MUNICÍPIO + ANO + INSTITUIÇÃO ===============================
agg_city_year_inst = {c: ["mean", "sum"] for c in value_cols}
agg_city_year_inst["Codigo_IBGE"] = "size"          # contagem de bolsistas

fato_mun_ano_inst = (
    df.groupby(lvl_city_year_inst, dropna=False)
      .agg(agg_city_year_inst)
      .rename(columns={"Codigo_IBGE": "Qtd_Bolsistas_Inst"})
      .pipe(flatten_cols)
)

fato_mun_ano_inst.to_excel("fato_mun_ano_inst.xlsx", index=False)

print("Arquivos salvos:")
print("- fato_municipio_ano.xlsx")
print("- fato_mun_ano_inst.xlsx")


Arquivos salvos:
- fato_municipio_ano.xlsx
- fato_mun_ano_inst.xlsx


In [2]:
import pandas as pd

# === 0. Carregar o arquivo base ===========================================
df = pd.read_excel("ICJ_CNPQ_2010_2023_LIMPO.xlsx")

# === 1. Definir chaves de agrupamento ====================================
lvl_city_year_inst_prog = [
    # chave município‑ano
    "Regiao_Geografica", "Unidade_Federacao", "Nome_Municipio",
    "Codigo_IBGE", "Ano", "Sigla_UF", "Pais",
    # dimensões adicionais
    "Instituição", "Programa", "Modalidade", "Area",
    "Grande_Area", "Linha_de_Fomento", "Origem_do_Recurso"
]

# === 2. Colunas numéricas a agregar ======================================
value_cols = [
    "Valor_Rs", "Valor_USS", "Benef_Modal/Ano",
    "Bolsa/Ano", "Auxílio/Ano"
]

# === 3. Função para “achatar” MultiIndex gerado pelo groupby =============
def flatten_cols(df_grouped):
    df_grouped.columns = [
        f"{col}_{func if func!='size' else ''}".rstrip("_")
        for col, func in df_grouped.columns
    ]
    return df_grouped.reset_index()

# === 4. Agregação ========================================================
agg_inst = {c: ["mean", "sum"] for c in value_cols}
agg_inst["Codigo_IBGE"] = "size"          # contagem de bolsistas

fato_mun_ano_inst = (
    df.groupby(lvl_city_year_inst_prog, dropna=False)
      .agg(agg_inst)
      .rename(columns={"Codigo_IBGE": "Qtd_Bolsistas_Inst"})
      .pipe(flatten_cols)
)

# === 5. Salvar ============================================================
fato_mun_ano_inst.to_excel("fato_mun_ano_inst.xlsx", index=False)

print("Arquivo gerado: fato_mun_ano_inst.xlsx")


Arquivo gerado: fato_mun_ano_inst.xlsx
